In [1]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Binance - Hyper price increase detection")
    .getOrCreate())

In [2]:
ohlcvDF = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "binance_data").option("startingOffsets", "earliest").load()



In [3]:
from pyspark.sql.functions import split, col
from pyspark.sql.types import DoubleType, LongType

#TCBUSD,1648234367780,43974.77000000

ohlcvDF = ohlcvDF.selectExpr("CAST(value AS STRING)").select(split("value",'\,').alias("fields")).withColumn("timestamp",col("fields").getItem(1).cast(LongType())).withColumn("symbol",col("fields").getItem(0)).withColumn("open",col("fields").getItem(2).cast(DoubleType())).drop("fields")


In [4]:
# from pyspark.sql.functions import concat, lit

# ohlcvDF.writeStream \
#         .format("console") \
#         .start() \
#         .awaitTermination()

22/03/29 08:50:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/np/7wstqrjj7h77xmr2lv_bt64h0000gn/T/temporary-020ff228-1561-4695-a1d0-11e18f927986. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/03/29 08:50:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1647938724128|BTCBUSD|41296.64|
|1647938725124|BTCBUSD|41292.35|
|1647938726104|BTCBUSD|41292.35|
|1647938727078|BTCBUSD|41292.35|
|1647938728117|BTCBUSD|41292.35|
|1647938729121|BTCBUSD|41292.35|
|1647938730132|BTCBUSD|41292.35|
|1647938731133|BTCBUSD|41292.35|
|1647938732108|BTCBUSD|41302.12|
|1647938733130|BTCBUSD|41302.13|
|1647938734127|BTCBUSD|41310.88|
|1647938735133|BTCBUSD|41310.87|
|1647938736130|BTCBUSD| 41303.8|
|1647938737123|BTCBUSD| 41303.8|
|1647938738094|BTCBUSD|41303.79|
|1647938739124|BTCBUSD|41303.79|
|1647938740131|BTCBUSD| 41303.8|
|1647938741127|BTCBUSD| 41303.8|
|1647938742122|BTCBUSD|41303.82|
|1647938743132|BTCBUSD|41303.82|
+-------------+-------+--------+
only showing top 20 rows

-------------------------------------------
Batch: 1
--------------------------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/Cellar/apache-spark/3.2.1/libexec/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql import Row
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

#https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/
class InfluxDBWriter:
    def __init__(self):
        self.t = 'C3GlPOdiT0dqAH9LH_Q8Z0vtXdkZ84DAkFE0phwEtgR_lbc7TYP4Sz6rTavhbNcLIjT9joY15Lyz4x1kkd30LA=='
        self.o = 'project' # database -> org
        self.b = 'binance' # table -> bucket
        self.client = InfluxDBClient(url="http://localhost:8086", token=self.t, org=self.o) # client creation
        self.write_api = self.client.write_api() # influx db write api

    def open(self, partition_id, epoch_id): # opens the connection to influx db
        print("Opened %d, %d" % (partition_id, epoch_id))
        return True

    def process(self, row): # writing the data into influx db table
        self.write_api.write(bucket=self.b, record=self._row_to_line_protocol(row))

    def close(self, error): #closing write apis and influxdb client
        self.write_api.__del__()
        self.client.__del__()
        print("Closed with error: %s" % str(error))

    def _row_to_line_protocol(self, row: Row): #converting the data into the format how it is stored in influx db
        # TODO map Row to LineProtocol
        p = Point("mess").tag("coin", row['symbol']).field("value", row['open']).time(row['timestamp'], WritePrecision.MS)
        return p
    
ohlcvDF.writeStream.foreach(InfluxDBWriter()).start().awaitTermination()
# we are taking the data and writing it into influx db using write stream API of spark

